In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from numpy import newaxis
import scipy.sparse as sps
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

In [ ]:
from pyfem.topo import Interval
from pyfem.poly import gll_points
from pyfem.sem import SEMhat
from pyfem.poly import eval_lagrange_d0 as eval_phi1d

In [ ]:
from poly import eval_P

In [ ]:
order = 1
semh = SEMhat(order)

N = 100
n_dofs = (order+1)*N

In [ ]:
L = 2.0

vertices  = np.linspace(0, L, N+1)
EtoV      = np.zeros((N, 2), dtype=np.int)
EtoV[:,0] = np.arange(N)
EtoV[:,1] = np.arange(N)+1

topo  = Interval()
xq = topo.ref_to_phys(vertices[EtoV], semh.xgll)
jacb_det = topo.calc_jacb(vertices[EtoV])[0]

# Restriction operator
# Removes contribution to dirch points from flux operators
R = sps.eye(n_dofs).tocsr()
R[0,0] = R[-1,-1] = 0.0

In [ ]:
h = xq[0,-1]-xq[0,0]

In [ ]:
# Make elem to dof map
EtoD = np.arange(N*(order+1))
EtoD = EtoD.reshape((N, -1))

dof_phys = xq.ravel()

# Averaging operator
rows = EtoD[:,[0,-1]].ravel()
cols = EtoV.ravel()
vals = np.ones_like(cols)

FtoD = sps.coo_matrix((vals, (rows, cols))).tocsr()
AVG = FtoD.dot(FtoD.T)/2.0

# Extract face DOFS
vals = np.ones(len(rows))
FD = sps.coo_matrix((vals, (rows, rows))).tocsr()
# Set face signs
vals[::2] = -1
SD = sps.coo_matrix((vals, (rows, rows))).tocsr()

# Jump operator
JUMP = FtoD.dot(SD.dot(FtoD).T)

In [ ]:
# Build Advection operator
C = sps.kron(sps.eye(N), semh.Ch).tocsr()

# Build full elemental mass matrix
x, w = topo.get_quadrature(order+1)
P = eval_phi1d(semh.xgll, x).T
G = sps.dia_matrix((w, 0), shape=(len(x), len(x)))
Bf = P.T.dot(G.dot(P))*jacb_det
Bfinv = np.linalg.inv(Bf)

# Using trick from book
V    = eval_P(order, semh.xgll).T
Vinv = np.linalg.inv(V)
Minv = V.dot(V.T)/jacb_det
Binv = sps.kron(sps.eye(N), Minv).tocsr()

print np.max(np.abs(Minv-Bfinv))
print np.max(np.abs(Minv.dot(Bf)-np.eye(Bf.shape[0])))
print np.max(np.abs(Bfinv.dot(Bf)-np.eye(Bf.shape[0])))

In [ ]:
T  = 0.4
dt = 0.001
nt = int(round(T/dt))
T, dt, nt

In [ ]:
# Problem setup

def calc_flux(u):
    return u*u/2.0

ul = 2.0
ur = 1.0
s  = (calc_flux(ur)-calc_flux(ul))/(ur-ul)

u0 = np.ones_like(dof_phys)*ur
u0[dof_phys<=L/2.0] = ul

ue = np.ones_like(u0)*ur
ue[dof_phys<=L/2.0+s*dt*nt] = ul

s, s*T+L/2.0

In [ ]:
# Time stepping matrix
lambda_max = s

FLUX = AVG+(lambda_max/2.0)*JUMP
F = Binv.dot(-C+R.dot(SD.dot(FD-FLUX)))

# Filter
ss = 10.0
a  = 36.0
eta = np.arange(order+1.0)/order
SIG = sps.dia_matrix((np.exp(-a*eta**ss),0),
                    shape=V.shape)
FILTER = sps.kron(sps.eye(N), V.dot(SIG.dot(Vinv)))

# Timestep on u'=g(u)
#g = lambda u: FILTER.dot(F.dot(calc_flux(u)))
g = lambda u: F.dot(calc_flux(u))

In [ ]:
# Slope limiter
x0 = (xq[:,0]+xq[:,-1])/2.0

In [ ]:
nv = np.arange(order+1)
gamma = 2.0/(2.0*nv+1)
G = sps.dia_matrix((1.0/np.sqrt(gamma),0),
                  shape=Vinv.shape)

G = G.dot(Vinv)

def minmod(a):
    
    l = a.shape[-1]
    s = np.sum(np.sign(a))/l
    if np.abs(s)==1:
        return s*np.min(np.abs(a))
    else:
        return 0.0

def slope_limit(u):
    
    us = u.reshape((-1,order+1)).T
    avg, slope = G.dot(us)[[0,1]]
    
    u = u.reshape((-1,order+1))
    for ielem in range(1,N-1):
        m = minmod(slope[ielem-1:ielem+1])
        u[ielem] = avg[ielem]+(xq[ielem]-x0[ielem])*m
        

In [ ]:
# Integrate with RK4
u  = u0.copy()

for k in range(nt):
        
    k1 = g(u)
    k2 = g(u+(dt/2.0)*k1)
    k3 = g(u+(dt/2.0)*k2)
    k4 = g(u+dt*k3)
    
    u = u+(dt/6.0)*(k1+2*k2+2*k3+k4)
    u[[0,-1]] = u0[[0,-1]]
    #u = FILTER.dot(u)
    u[[0,-1]]  = u0[[0,-1]]

    slope_limit(u)
    
u_rk4 = u
np.max(np.abs(u-ue))

In [ ]:
plt.plot(dof_phys, u_rk4)
plt.plot(dof_phys, ue, 'g--')